# Automated Range Scan

The parameters in this notebook are tuned for a Raspberry Pi 4 with the following configuration:

- Fixed core voltage of 0.96 V
- Fixed frequency of 1.8 GHz (`force_turbo=1`)

In [253]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'

In [254]:
if scope is not None:
    scope.dis()
if target is not None:
    target.dis()

%run "../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 56263717                  to 67260350                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 398280086                 to 96000000                 
scope.clock.adc_rate                     changed from 398280086.0               to 96000000.0        

In [255]:
def reboot_flush():
    scope.io.nrst = False # GLOBAL_EN is wired up to nRST
    time.sleep(0.05)
    scope.io.nrst = "high_z"
    # Wait for the Pi to boot
    time.sleep(30)
    # Flush garbage too
    target.flush()

In [256]:
cw.set_all_log_levels(cw.logging.DEBUG)
for _ in range(1): # NOTE: Test for reliability with repetitions
    reboot_flush() # Test that resets are working
    scope.arm()
    target.simpleserial_write("g", bytearray([]))
    scope.capture() # Counterintuitively, this needs to be called *after* sending the command
    val = target.simpleserial_read_witherrors('r', 4, timeout=2000) # The timeout needs to be large enough to avoid strange errors
    print(val)
    valid = val['valid']
    if valid:
        response = val['payload']
        raw_serial = val['full_response']
        error_code = val['rv']
    else:
        raise Exception("not valid")

{'valid': True, 'payload': CWbytearray(b'00 e1 f5 05'), 'full_response': CWbytearray(b'00 72 04 00 e1 f5 05 13 00'), 'rv': bytearray(b'\x00')}


In [257]:
scope.clock.clkgen_freq = 100E6 # 100 MHz
scope.glitch.clk_src = "clkgen"

`enable_only` sets the CW to perform a pure voltage glitching attack asynchronous to the target's clock. This means the following:

- `repeat` = `width`
- `ext_offset` = `offset`

In [258]:
scope.glitch.output = "enable_only" # Pure voltage glitching: repeat = width and ext_offset = offset
scope.io.glitch_lp = True # Both need to be enabled for this to work reliably
scope.io.glitch_hp = True
scope.io

tio1         = serial_rx
tio2         = serial_tx
tio3         = high_z
tio4         = high_z
pdid         = high_z
pdic         = high_z
nrst         = high_z
glitch_hp    = True
glitch_lp    = True
extclk_src   = hs1
hs2          = clkgen
target_pwr   = True
tio_states   = (1, 1, 0, 0)
cdc_settings = bytearray(b'\x00\x00\x00\x00')

In [259]:
gc = cw.GlitchController(groups=["success", "reset", "soft", "normal"], parameters=["repeat", "ext_offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='soft count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='repeat setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

In [260]:
gc.glitch_plot(plotdots={"success": "+g", "reset": "xr", "soft": None, "normal": None}, bufferlen=int(10E6))

/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:DynamicMap   []
   :Overlay
      .Points.I  :Points   [repeat,ext_offset]
      .Points.II :Points   [repeat,ext_offset]

In [261]:
# Ideas:
# For each repeat, binary search a small enough crash boundary (using n averages) and then try glitching there?
# Since everything (voltage & freq.) should be fixed, what about binary searching common crash boundary, even if large-ish and then doing the exhaustive search?
# This would allow figuring out a nice range to step through for repeat, and then ext_offset can be computed with time (and basically just needs to hit the computation)

In [262]:
import struct
import random

# Disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)
#cw.set_all_log_levels(cw.logging.WARNING)

scope.adc.timeout = 1 # The simpleserial-glitch emulator takes a bit to respond

#reboot_flush()
#total_successes = 0
successes = 0
normals = 0
resets = 0

# New glitch boundary searching algorithm
offset_time_min = 0.01 # Earliest time (in seconds) when to apply the glitch after the trigger 
offset_time_max = 0.1 # Latest time (in seconds) when to apply the glitch after the trigger
repeat = 1700 # Starting factor of repeat (glitch width)
backoff_factor = 10 # Starting factor to back off the width with
noop_threshold = 300 # How many no-op results to observe before trying to advance again
success_threshold = 10 # How many successful glitches are considered a success

#scope.glitch.ext_offset = scope.clock.clkgen_freq * offset_time
print(f"repeat: {repeat}, backoff_factor: {backoff_factor}")

while True:
    scope.glitch.repeat = repeat
    scope.glitch.ext_offset = scope.clock.clkgen_freq * random.uniform(offset_time_min, offset_time_max)
    gc.parameter_values = [scope.glitch.repeat, scope.glitch.ext_offset]

    # Rate limit the attack to let the caps charge in
    # between (ensure independence between attempts)
    # time.sleep(1)
    
    target.flush()
    scope.arm() # This starts the capture
    target.simpleserial_write("g", bytearray([])) # Run the loop on the Pi
    scope.capture() # This ends the capture
    scope.io.vglitch_reset()

    # Success, but was it a glitch or not?
    val = target.simpleserial_read_witherrors('r', 4, timeout=2000)
    if val['valid'] is False:
        # This might already be indicative enough of a full crash
        
        # Checking the return of scope.capture() is very unreliable, since it picks up
        # various ADC sample size errors, probably due to the variable runtime.
        # Instead, this check receiving garbage/timeout should be the decider for
        # a soft/hard reset -> similar to above, hard reset if target doesn't respond
        # to version ACK

        time.sleep(3) # Wait for the application to recover (if application crash)
        target.flush()
        target.simpleserial_write("v", bytearray([])) # Request version to check liveness
        if target.simpleserial_wait_ack() is not None: # If live, this was a soft crash
            # Soft resets are not tracked under the "reset" category
            print("Soft reset (application crash)")
            gc.add("soft")
            continue
        
        # Hard crash
        print("Hard reset (system hang)")
        gc.add("reset")
        reboot_flush()
        repeat -= backoff_factor # Back off from even a single hard reset
        normals = 0 # Reset no-op count
        print("Backing off...")
        print(f"repeat: {repeat}, backoff_factor: {backoff_factor}")
    else:
        gcnt = struct.unpack("<I", val['payload'])[0]
        if gcnt != 100_000_000: # Check if we got to 100 million or not
            # Successful glitch!
            print("🍓 Success: {}, {}", gcnt, gc.parameter_values)
            gc.add("success")
            successes += 1
            normals = 0 # Reset no-op count
        else:
            # Nothing remarkable happened...
            gc.add("normal")
            normals += 1

        if successes == success_threshold:
            break # We've glitched enough

        if normals > noop_threshold:
            print("Nothing is happening, let's be more aggressive...")
            repeat += backoff_factor // 10 # Advance opportunistically
            print(f"repeat: {repeat}, backoff_factor: {backoff_factor}")
            normals = 0

print("Done glitching")
cw.set_all_log_levels(cw.logging.WARNING)

repeat: 1700, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1690, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1680, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1670, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1660, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1650, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1640, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1630, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1620, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1610, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1600, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1590, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1580, backoff_factor: 10
Hard reset (system hang)
Backing off...
repeat: 1570, backoff_factor: 10
Hard reset (system

KeyboardInterrupt: 

In [ ]:
# Old algorithm (original from CW tutorials)

#num_tries = 1 # increase to get better glitch stats
#gc.set_range("tries", 1, num_tries)

# Broad scan (no idea what happens)
#gc.set_range("ext_offset", 100, 100) # TODO: To be tuned, for now just try to hit the loop
#gc.set_range("repeat", 1, 100) # TODO: Hone this in with Manual.ipynb, or maybe automatically? -> Need to find the crash boundary
#gc.set_global_step([8, 4, 2, 1]) # TODO: Hone in on crash boundary

# for glitch_setting in gc.glitch_values():
#     print(glitch_setting)
#     scope.glitch.repeat = glitch_setting[0]
#     scope.glitch.ext_offset = glitch_setting[1]
#     #if glitch_setting[2] == 1:
#     #    total_successes += successes
#     #    if (successes > 0):
#     #        print("successes = {}, resets = {}, repeat = {}, ext_offset = {}".format(successes, resets, scope.glitch.repeat, scope.glitch.ext_offset))
#     #        total_successes += successes
#     #    successes = 0
#     #    resets = 0
#         #if total_successes > MAX_SUCCESSES:
#         #    break
#     target.flush()
    
#     if scope.adc.state:
#         # can detect crash here (fast) before timing out (slow)
#         print("Trigger still high!")
#         gc.add("reset")
#         # Device is slow to boot?
#         reboot_flush()
#         resets += 1

#     scope.arm() # This starts the capture
#     target.simpleserial_write("g", bytearray([])) # This is probably too fast, the trigger gets missed?
#     ret = scope.capture() # This ends the capture
#     scope.io.vglitch_reset()
    
#     if ret:
#         print('Timeout - no trigger')
        
#         gc.add("reset")
#         resets += 1

#         break # TODO: Remove

#         # Device is slow to boot?
#         reboot_flush()

#     else:
#         val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10, timeout=50)
#         if val['valid'] is False:
#             gc.add("reset")
#             reboot_flush()
#             resets += 1
#         else:
#             gcnt = struct.unpack("<I", val['payload'])[0]

#             if gcnt != 10000000: #for loop check
#                 gc.add("success")
#                 #print((scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
#                 successes += 1
#             else:
#                 gc.add("normal")

#             # TODO: Let the caps charge in between?
#             time.sleep(1)

In [263]:
scope.dis()
target.dis()